In [1]:
# Configurações de visualização
%matplotlib inline
%config inlineBackend.figure_formats = ['svg']

# Importação das Bibliotecas

# Manipulação
import pandas as pd
import re

import joblib

# Configurações de visualização do dataset
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 90)

# 4.1 - Pré-processamento para os Dados de Teste

Exatamente! Você pode treinar o seu modelo com os dados de treinamento (cars_training) e excluir a coluna "id" desses dados, se ela não for relevante para o treinamento do modelo.

No entanto, quando for criar o resultado final conforme solicitado usando os dados de teste (cars_test), é importante manter a coluna "id". Isso ocorre porque o resultado final deve incluir as identificações corretas das instâncias no conjunto de teste.

Portanto, resumindo:

Treinamento do modelo:

Use os dados de treinamento (cars_training).
Exclua a coluna "id" dos dados de treinamento, se ela não for relevante para o treinamento do modelo.
Criação do resultado final:

Use os dados de teste (cars_test).
Mantenha a coluna "id" nos dados de teste.
Faça as previsões do modelo nos dados de teste, incluindo a coluna "id".
Crie o resultado final com as colunas "id" e "preco" (previsões do modelo).
Certifique-se de seguir as instruções detalhadas do projeto para garantir que você esteja cumprindo todos os requisitos específicos de formatação e entrega do resultado final.

In [2]:
# Dados de Teste
df_test = pd.read_csv('cars_test.csv', encoding='utf-16le', sep='\t')

In [3]:
# Substituir os valores NaNs como a segunda categória nas colunas mencionadas
df_test['garantia_de_fábrica'] = df_test['garantia_de_fábrica'].fillna("Sem Garantia de fábrica")
df_test['revisoes_dentro_agenda'] = df_test['revisoes_dentro_agenda'].fillna("Revisões feitas fora da agenda do carro")
df_test['revisoes_concessionaria'] = df_test['revisoes_concessionaria'].fillna("Revisões feitas fora da concessionária")
df_test['veiculo_único_dono'] = df_test['veiculo_único_dono'].fillna("Mais de um dono")
df_test['veiculo_licenciado'] = df_test['veiculo_licenciado'].fillna("Não Licenciado")
df_test['ipva_pago'] = df_test['ipva_pago'].fillna("IPVA Não Pago")
df_test['dono_aceita_troca'] = df_test['dono_aceita_troca'].fillna("Não Aceita Troca")

In [4]:
# Não precisa excluir a coluna "id" no dataset de test pois vamos utilizar ela para identificar as previsões
#df_test.drop('id', axis=1, inplace=True)

# Excluir a coluna  "veiculo_alienado"
df_test.drop('veiculo_alienado', axis=1, inplace=True)

# Excluir a coluna  "elegivel_revisao"
df_test.drop('elegivel_revisao', axis=1, inplace=True)

# Excluir a coluna  "ano_modelo"
df_test.drop('ano_modelo', axis=1, inplace=True)

# Excluir os valores nulos da coluna "num_fotos"
df_test.dropna(subset=['num_fotos'], inplace=True)

In [5]:
# Trasformar a colunas  "num_fotos" para int64
df_test['num_fotos'] = df_test['num_fotos'].astype('int64')

In [6]:
# Função para extrair a cilindrada da coluna versão
def cilindrada(versao):
    match = re.search(r'\d+(\.\d+)?', versao)
    if match:
        return float(match.group())
    else:
        return None

# Aplicar a função para extrair a cilindrada da coluna "versao" e criar a nova coluna "cilindrada"
df_test['cilindrada'] = df_test['versao'].apply(cilindrada)

In [7]:
# Função para extrair o tipo de combustível da coluna versão
def combustivel(valor):
    padrao = r'\b(FLEX|GASOLINA|DIESEL)\b'
    matches = re.findall(padrao, valor, flags=re.IGNORECASE)
    if matches:
        return matches[0].capitalize()
    else:
        return None

# Aplicar a função na coluna 'versao' e criar uma nova coluna 'tipo_combustivel'
df_test['tipo_combustivel'] = df_test['versao'].apply(combustivel)

In [8]:
# Classificar os valores restantes como "Outros"
df_test['tipo_combustivel'] = df_test['tipo_combustivel'].fillna('Outros')

In [9]:
# Excluir a variável versao
df_test.drop("versao", axis=1, inplace=True)

In [10]:
# Remover linhas com valores nulos
df_test = df_test.dropna()

In [11]:
# Agrupar os dados por marca e modelo e contar o número de modelos únicos para cada marca
modelos_por_marca = df_test.groupby('marca')['modelo'].nunique().reset_index()

# Renomear a coluna resultante
modelos_por_marca = modelos_por_marca.rename(columns={'modelo': 'modelos_por_marca'})

# Criar um dicionário de mapeamento usando a coluna de marcas como chave e o número de modelos como valor
map1 = dict(zip(modelos_por_marca['marca'], modelos_por_marca['modelos_por_marca']))

# Aplicar o mapeamento para criar a nova coluna "modelos_por_marca"
df_test['modelos_por_marca'] = df_test['marca'].map(map1)

In [12]:
# Agrupar os dados por tipo e modelo e contar o número de modelos únicos para cada tipo
modelos_por_tipo = df_test.groupby('tipo')['modelo'].nunique().reset_index()

# Renomear a coluna resultante
modelos_por_tipo = modelos_por_tipo.rename(columns={'modelo': 'modelos_por_tipo'})

# Criar um dicionário de mapeamento usando a coluna tipo como chave e o número de modelos como valor
map2 = dict(zip(modelos_por_tipo['tipo'], modelos_por_tipo['modelos_por_tipo']))

# Aplicar o mapeamento para criar a nova coluna "modelos_por_tipo"
df_test['modelos_por_tipo'] = df_test['tipo'].map(map2)

In [13]:
# Agrupar os dados por ano de fabricação e modelo e contar o número de modelos únicos para cada
modelos_por_anofabricacao = df_test.groupby('ano_de_fabricacao')['modelo'].nunique().reset_index()

# Renomear a coluna resultante
modelos_por_anofabricacao = modelos_por_anofabricacao.rename(columns={'modelo': 'modelos_por_anofabricacao'})

# Criar um dicionário de mapeamento usando a coluna ano_de_fabricacao como chave e o número de modelos como valor
map3 = dict(zip(modelos_por_anofabricacao['ano_de_fabricacao'], modelos_por_anofabricacao['modelos_por_anofabricacao']))

# Aplicar o mapeamento para criar a nova coluna "modelos_por_anofabricacao"
df_test['modelos_por_anofabricacao'] = df_test['ano_de_fabricacao'].map(map3)

In [14]:
# Agrupar os dados por estado e modelo e contar o número de modelos únicos para cada estado
modelos_por_estado = df_test.groupby('estado_vendedor')['modelo'].nunique().reset_index()

# Renomear a coluna resultante
modelos_por_estado = modelos_por_estado.rename(columns={'modelo': 'modelos_por_estado'})

# Criar um dicionário de mapeamento usando a coluna de estados como chave e o número de modelos como valor
map4 = dict(zip(modelos_por_estado['estado_vendedor'], modelos_por_estado['modelos_por_estado']))

# Aplicar o mapeamento para criar a nova coluna "modelos_por_estado"
df_test['modelos_por_estado'] = df_test['estado_vendedor'].map(map4)

In [15]:
# Agrupar os dados por anunciante e modelo e contar o número de modelos únicos para cada anunciante
modelos_por_anunciante = df_test.groupby('anunciante')['modelo'].nunique().reset_index()

# Renomear a coluna resultante
modelos_por_anunciante = modelos_por_anunciante.rename(columns={'modelo': 'modelos_por_anunciante'})

# Criar um dicionário de mapeamento usando a coluna anunciante como chave e o número de modelos como valor
map5 = dict(zip(modelos_por_anunciante['anunciante'], modelos_por_anunciante['modelos_por_anunciante']))

# Aplicar o mapeamento para criar a nova coluna "modelos_por_anunciante"
df_test['modelos_por_anunciante'] = df_test['anunciante'].map(map5)

In [16]:
# Mudando a ordem das colunas
df_test = df_test.reindex(columns=['id', 'num_fotos', 'marca', 'modelo', 'modelos_por_marca', 'modelos_por_tipo',
                                            'modelos_por_anofabricacao','modelos_por_estado', 'modelos_por_anunciante',
                                              'ano_de_fabricacao', 'hodometro', 'cambio','tipo', 'blindado', 'cor',
                                                'num_portas', 'cilindrada', 'tipo_combustivel', 'tipo_vendedor',
                                                'cidade_vendedor', 'estado_vendedor', 'anunciante', 'entrega_delivery',
                                                  'troca', 'dono_aceita_troca','veiculo_único_dono', 'ipva_pago',
                                                    'veiculo_licenciado', 'revisoes_concessionaria', 'revisoes_dentro_agenda',
                                                    'garantia_de_fábrica'])

In [17]:
df_test.head()

,id,num_fotos,marca,modelo,modelos_por_marca,modelos_por_tipo,modelos_por_anofabricacao,modelos_por_estado,modelos_por_anunciante,ano_de_fabricacao,hodometro,cambio,tipo,blindado,cor,num_portas,cilindrada,tipo_combustivel,tipo_vendedor,cidade_vendedor,estado_vendedor,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,ipva_pago,veiculo_licenciado,revisoes_concessionaria,revisoes_dentro_agenda,garantia_de_fábrica
0,13518783164498355150900635905895481162,8,NISSAN,VERSA,8,123,98,332,357,2021,20258.0,Manual,Hatchback,N,Branco,4,1.6,Outros,PF,Rio de Janeiro,São Paulo (SP),Pessoa Física,False,False,Aceita troca,Mais de um dono,IPVA pago,Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Garantia de fábrica
1,299896161723793383246777788797566040330,18,FIAT,STRADA,18,123,98,10,230,2021,53045.0,Manual,Picape,N,Branco,2,1.4,Flex,PJ,Palmas,Amazonas (AM),Loja,False,False,Aceita troca,Único dono,IPVA pago,Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica
2,316180649972302128246133616457018378621,8,AUDI,Q5,18,131,152,89,357,2018,32486.0,Automática,Utilitário esportivo,N,Branco,4,2.0,Gasolina,PF,Goiânia,Goiás (GO),Pessoa Física,False,False,Aceita troca,Mais de um dono,IPVA pago,Não Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica
3,222527157104148385909188217274642813298,16,CHEVROLET,CRUZE,26,269,130,332,230,2019,24937.0,Automática,Sedã,N,Cinza,4,1.4,Flex,PJ,Presidente Prudente,São Paulo (SP),Loja,False,False,Aceita troca,Único dono,IPVA pago,Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica
4,160460343059850745858546502614838368036,8,FORD,ECOSPORT,14,269,130,332,230,2019,62074.0,Automática,Sedã,N,Branco,4,1.5,Flex,PJ,Limeira,São Paulo (SP),Loja,False,False,Aceita troca,Mais de um dono,IPVA pago,Licenciado,Revisões feitas fora da concessionária,Revisões feitas fora da agenda do carro,Sem Garantia de fábrica


In [18]:
# Tamanho do dataset
df_test.shape

(9800, 31)

In [19]:
# Carregar o modelo do disco
modeloxgb_carregado = joblib.load('modelo_xgb.pkl')

In [20]:
# Aplicar o pré-processamento aos dados de teste
df_test2 = modeloxgb_carregado.named_steps['preprocessor'].transform(df_test)

# Fazer previsões com o modelo XGBoost
Y_pred = modeloxgb_carregado.named_steps['model'].predict(df_test2)


In [21]:
Y_pred.shape

(9800,)

In [22]:
# Criar DataFrame com as colunas "id" e "preco"
predicted = pd.DataFrame({
    'id': df_test['id'],  # Manter a coluna "id" do conjunto de teste
    'preco': Y_pred  # Previsões do modelo
})

In [23]:
predicted.head()

,id,preco
0,13518783164498355150900635905895481162,75248.281250
1,299896161723793383246777788797566040330,74405.585938
2,316180649972302128246133616457018378621,219513.562500
3,222527157104148385909188217274642813298,120956.804688
4,160460343059850745858546502614838368036,93907.789062
